<a href="https://colab.research.google.com/github/umar-alsammarraie/architecture-dataset/blob/main/code/architecture_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
# Set the random seed for reproducibility
tf.random.set_seed(42)

In [7]:
# Set the path to your dataset directory
dataset_dir = '/content/sample_data/'

# Set the image size and batch size
image_size = (224, 224)
batch_size = 32

# Create an ImageDataGenerator for data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Split the dataset into training and validation sets
)


In [9]:
# Load and preprocess the training set
print('Train data:')
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    class_mode='categorical',
    batch_size=batch_size,
    subset='training'  # Use the subset for training
)
# Load and preprocess the validation set
print('Validation data:')
val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use the subset for validation
)

Train data:
Found 0 images belonging to 0 classes.
Validation data:
Found 0 images belonging to 0 classes.


In [10]:
# Build the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

In [ ]:
# Predict a class of a specific image
from PIL import Image
import numpy as np

# Load the image you want to classify
image_path = '/path/to/test/image.jpg'  # Replace with the path to your test image
image = Image.open(image_path)
image = image.resize(image_size)  # Resize the image to match the input size of the model
image = np.array(image) / 255.0  # Normalize pixel values between 0 and 1
image = np.expand_dims(image, axis=0)  # Add an extra dimension for batch size

# Make predictions on the image
predictions = model.predict(image)
class_index = np.argmax(predictions[0])
class_label = train_generator.class_indices  # Get the mapping of class indices to labels
class_label = {v: k for k, v in class_label.items()}  # Reverse the mapping

# Print the predicted class label
print("Predicted architecture style:", class_label[class_index])